# Simon's Algorithm

It consists of running the circuit several times followed by a Post-processing step.

In [1]:
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit
import numpy as np

def simon_function(s: str):
    """
    Create a QC implementin a query gate for Simon's Algorithm
    """
    # QC has 2n qubits for n = len(s)
    n = len(s)
    qc = QuantumCircuit(2 * n)
    # Random permutation of all n bit strings.
    pi = np.random.permutation(2**n)

    # Deine a query gate explicitly.
    query_gate = np.zeros((4**n, 4**n))
    for x in range(2**n):
        for y in range(2**n):
            z = y ^ pi[min(x, x ^ int(s, 2))]
            query_gate[x + 2**n * z, x + 2**n * y] = 1

    qc.unitary(query_gate, range(2 * n))
    return qc

In [2]:
from qiskit_aer import AerSimulator
from qiskit import ClassicalRegister

def simon_measurements(problem: QuantumCircuit, k: int):
    """
    Quantum part of Simon's Algorithm. Given a `QuantumCircuit
    implements f get `k` measurements to be post-processed later.
    """
    n = problem.num_qubits // 2

    qc = QuantumCircuit(2 * n, n)
    qc.h(range(n))
    qc.compose(problem, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))

    result = AerSimulator().run(qc, shots=k, memory=True).result()
    return result.get_memory()


In [3]:
display(simon_measurements(simon_function("11011"),k=12))

['11000',
 '11100',
 '11011',
 '01110',
 '01001',
 '00011',
 '10110',
 '01101',
 '00000',
 '01110',
 '10010',
 '11111']

In [4]:
import numpy as np
import galois

def simon_algorithm(problem: QuantumCircuit):
    """
    Given a `QuantumCircuit` that implements a query gate for Simon problem, return the hidden string `s`.
    """

    # Quantum part: run the circuit defined previously k times and gather the measurement results.
    # Replace +10 by +r for any nonnegative integer r depending on desired confidence.

    measurements = simon_measurements(problem, k=problem.num_qubits // 2 + 10)
    print("Measurement results:")
    display(measurements)

    # Classical post-processing:

    # 1. Convert measurements of form '11101' to 2D-array of integers
    matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int)

    # 2. Interpret matrix as using arithmetic mod 2, and find null space
    null_space = galois.GF(2)(matrix).null_space()
    print("Null space:")
    display(null_space)

    # 3. Convert back to a string
    print("Guess for hidden string s:")
    if len(null_space) == 0:
        # No non-trivial solution; `s` is all-zeros
        return "0" * len(measurements[0])
    return "".join(np.array(null_space[0]).astype(str))

ModuleNotFoundError: No module named 'galois'

In [ ]:
display(simon_algorithm(simon_function("10011")))

Measurement Results:


['00100',
 '11001',
 '10101',
 '11101',
 '00111',
 '10001',
 '11101',
 '00111',
 '01000',
 '01100',
 '11010',
 '11110',
 '11001',
 '11001',
 '11010']

NameError: name 'galois' is not defined